In [ ]:
# --- Cell 1 ---
%matplotlib inline
import numpy as np
import random

import jax.numpy as jnp
from jax import jacfwd, jit, vmap

import proxsuite
from cyipopt import minimize_ipopt
from scipy.stats import norm
from scipy.interpolate import interp1d

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="vpython")

In [ ]:
# --- Cell 2 ---
tf = 5.              # total time
dt = 0.05            # simulation time step
Ns = 4               # dimension of state
Nc = 2               # dimension of control
S = round(tf/dt) + 1 # number of knot points
print(f"S = {S}")

# Parameters
g = 9.81 # gravity (m/s^2)
m1 = 1.  # mass of link 1 (kg)
m2 = 1.  # mass of link 2 (kg)
l1 = 0.5 # length of link 1 (m)
l2 = 0.5 # length of link 2 (m)

In [ ]:
# --- Cell 3 ---
# Cost
Q = np.diag([500., 500., 10., 10.]) 
R = 0.01 * np.eye(Nc)
QF = 2000. * np.eye(Ns)

Let's get the results of trajectory optimization ($\bar{x}, \bar{u}$)!

In [ ]:
# --- Cell 4 ---
results = pd.read_csv('results.csv')
q1_bar = results['q1'].values[::2]
q2_bar = results['q2'].values[::2]
q1_dot_bar = results['q1_dot'].values[::2]
q2_dot_bar = results['q2_dot'].values[::2]
u1_bar = results['u1'].values[::2]
u2_bar = results['u2'].values[::2]

xb = np.column_stack((q1_bar, q2_bar, q1_dot_bar, q2_dot_bar))
ub = np.column_stack((u1_bar, u2_bar))

xb = jnp.array(xb).T 
ub = jnp.array(ub).T

In [ ]:
# --- Cell 5 ---
def M(q):
    q1, q2 = q[0,0], q[1,0]
    A = jnp.array([[l1**2 * m1 + l2**2 * m2 + l1**2 * m2 + 2 * l1 * m2 * l2 * jnp.cos(q2), l2**2 * m2 + l1 * m2 * l2 * jnp.cos(q2)],
                  [                               l2**2 * m2 + l1 * m2 * l2 * jnp.cos(q2),                              l2**2 * m2]])

    return A

def C(q, q_dot):
    q1, q2 = q[0,0], q[1,0]
    q1_dot, q2_dot = q_dot[0,0], q_dot[1,0]
    A = jnp.array([[-2 * q2_dot * l1 * m2 * l2 * jnp.sin(q2), -q2_dot * l1 * m2 * l2 * jnp.sin(q2)],
                  [      q1_dot * l1 * m2 * l2 * jnp.sin(q2),                                    0]])
    return A

def G(q):
    q1, q2 = q[0,0], q[1,0]
    A = jnp.array([[-g * m1 * l1 * jnp.sin(q1) - g * m2 * (l1 * jnp.sin(q1) + l2 * jnp.sin(q1+q2))],
                  [                         -g * m2 * l2 * jnp.sin(q1+q2)                         ]])
    return A

In [ ]:
# --- Cell 6 ---
def dynamics(x, u):
    q     = jnp.block([[x[0,0], x[1,0]]]).T
    q_dot = jnp.block([[x[2,0], x[3,0]]]).T

    lhs = M(q)
    rhs = u - C(q,q_dot) @ q_dot + G(q)
    q_ddot = jnp.linalg.solve(lhs, rhs)

    return jnp.block([[q_dot], [q_ddot]])

In [ ]:
# --- Cell 7 ---
# Runge-Kutta 4th Order integration
def rk4(x, u):
    f1 = dynamics(x, u)
    f2 = dynamics(x + f1 * dt/2, u)
    f3 = dynamics(x + f2 * dt/2, u)
    f4 = dynamics(x + f3 * dt, u)
    
    return x + dt/6 * (f1 + 2*f2 + 2*f3 + f4)

# Linearization
def linearize(x_bar, u_bar):
    A = (jacfwd(rk4, argnums=0)(x_bar, u_bar)).reshape(Ns,Ns)
    B = (jacfwd(rk4, argnums=1)(x_bar, u_bar)).reshape(Ns,Nc)
    
    return A, B

In [ ]:
# --- Cell 8 ---
# Cost function
def cost(xs, us):
    c = 0.5 * xs[-1].T @ QF @ xs[-1]
    for k in range(S-1):
        c += 0.5 * xs[k].T @ Q @ xs[k]
        c += 0.5 * us[k].T @ R @ us[k]
    return c[0][0]

# **3. Controlling via TVLQR**

1. Write a function in Python called $\texttt{my\_controller()}$ that takes as input the current state
of the system, and outputs the controls to be applied to the system. You should linearize
around the trajectory that the previous step outputs.

In [ ]:
# --- Cell 9 ---
def my_controller(current_x, add_noise=False):
    # Initializations
    Ks = [jnp.zeros((Nc, Ns))] * (S-1)
    Ps = [jnp.zeros((Ns, Ns))] * S

    if(add_noise):
        Q = np.diag([500., 500., 10., 10.]) 
        R = 20. * np.eye(Nc)
        QF = 2000. * np.eye(Ns)
        
        # Generate Gaussian Noise
        mu = 0
        # sigma = np.array([0.01, 0.01, 0.1, 0.1]).reshape(-1, 1) # low noise level
        # sigma = np.array([0.4, 0.4, 0.5, 0.5]).reshape(-1, 1)   # intermediate noise level
        sigma = np.array([0.9, 0.9, 1., 1.]).reshape(-1, 1)     # high noise level
    
    # 1. Backward Pass: Riccati Recursion
    Ps[S-1] = QF
    for k in range(S-2, -1, -1):
        x_bar = xb[:Ns, k].reshape(Ns,-1)
        u_bar = ub[:Nc, k].reshape(Nc,-1)
        
        A, B = linearize(x_bar, u_bar)
        lhs = (R + B.T @ Ps[k+1] @ B)
        rhs = B.T @ Ps[k+1] @ A
        Ks[k] = jnp.linalg.solve(lhs, rhs)
        Ps[k] = Q + A.T @ Ps[k+1] @ (A - B @ Ks[k])

    # 2. Forward Pass: Simulation with Feedback Control
    x = np.copy(current_x)
    xs = [jnp.copy(x)]
    us = [jnp.zeros((Nc, 1))]

    for k in range(S-1):
        x_bar = xb[:Ns, k].reshape(Ns,1)
        u_bar = ub[:Nc, k].reshape(Nc,1)

        if(add_noise):
            noise = np.random.normal(mu, sigma, (Ns, 1))
            observation = x + noise
            
            x_bar = xb[:Ns, k].reshape(Ns,1)
            u_bar = ub[:Nc, k].reshape(Nc,1)
    
            u = u_bar - Ks[k] @ (observation - x_bar)
        else:
            u = u_bar - Ks[k] @ (x - x_bar)
    
        u = np.clip(u, -10., 10.)
        x = rk4(x, u)
        
        xs.append(jnp.copy(x))
        us.append(jnp.copy(u))
        
    print(f"Total cost = {cost(xs, us)}")
    
    return xs, us

Let's try the controller with different initial states.

In [ ]:
# --- Cell 10 ---
# Initial state
x0 = jnp.array([[0., 0., 0., 0.]]).T              # 1
# x0 = jnp.array([[jnp.pi/2, -jnp.pi/2, 0., 0.]]).T # 2
# x0 = jnp.array([[-jnp.pi/3, jnp.pi/4, 0., 0.]]).T # 3
# x0 = jnp.array([[jnp.pi, 0, 0., 0.]]).T           # 4
# x0 = jnp.array([[jnp.pi, -2, 0., 0.]]).T          # 5
# x0 = jnp.array([[jnp.pi/2.5, -2.3, 3., -2.]]).T   # 6
# x0 = jnp.array([[0.3, 0.2, 0., 0.]]).T            # 7

# Results
xs, us = my_controller(x0)
xs = np.array(xs).reshape(-1,Ns)
us = np.array(us).reshape(-1,Nc)

Let's plot the trajectory of double pendulum!

In [ ]:
# --- Cell 11 ---
# Trajectory Simulation
q1     = xs[:, 0]
q2     = xs[:, 1]
q1_dot = xs[:, 2]
q2_dot = xs[:, 3]
u1     = us[:, 0]
u2     = us[:, 1]

# Link 1
x1 = l1 * np.sin(q1)
y1 = -l1 * np.cos(q1)

# Link 2
x2 = x1 + l2 * np.sin(q1 + q2)
y2 = y1 - l2 * np.cos(q1 + q2)

frames = 10
indices = np.linspace(0, len(q1)-1, frames, dtype=int)
colors = plt.cm.summer(np.linspace(0, 1, frames))
    
plt.figure(figsize=(10, 8))
plt.title('Double Pendulum Swing-up Trajectory')
plt.axis('equal')

plt.plot([-0.5, 0.5], [0, 0], color='black', lw=2.5) # black base-link
for i, idx in enumerate(indices):
    plt.plot([0, x1[idx], x2[idx]], [0, y1[idx], y2[idx]], color=colors[i], marker='o', alpha=0.7, linewidth=2) # double pendulum
plt.plot(x2, y2, 'k--', alpha=0.3) # trajectory trace
plt.xlabel('x (m)')
plt.ylabel('y (m)')

plt.text(x2[0], y2[0] - 0.07, 'start', color=colors[0], ha='center', fontweight='bold')  # 'start' text
plt.text(x2[-1], y2[-1] + 0.05, 'end', color=colors[-1], ha='center', fontweight='bold') # 'end' text

plt.savefig('3-trajectory.png', dpi=300)
plt.show()

In [ ]:
# --- Cell 12 ---
# Animation
states = np.array(xs).T
S = states.shape[1]

fig, ax = plt.subplots(figsize=(8, 8))

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.grid(True)
ax.set_title('Controlling via TVLQR ($x_0 = [0, 0, 0, 0]$, $x_{des} = [π, 0, 0, 0]$)')
# ax.set_title('Controlling via TVLQR ($x_0 = [π/2, -π/2, 0, 0]$, $x_{des} = [π, 0, 0, 0]$)')
# ax.set_title('Controlling via TVLQR ($x_0 = [-π/3, -π/4, 0, 0]$, $x_{des} = [π, 0, 0, 0]$)')
# ax.set_title('Controlling via TVLQR ($x_0 = [π, 0, 0, 0]$, $x_{des} = [π, 0, 0, 0]$)')
# ax.set_title('Controlling via TVLQR ($x_0 = [π, -2, 0, 0]$, $x_{des} = [π, 0, 0, 0]$)')
# ax.set_title('Controlling via TVLQR ($x_0 = [π/2.5, -2.3, 3, -2]$, $x_{des} = [π, 0, 0, 0]$)')
# ax.set_title('Controlling via TVLQR ($x_0 = [0.3, 0.2, 0, 0]$, $x_{des} = [π, 0, 0, 0]$)')

ax.plot([-0.5, 0.5], [0, 0], color='black', lw=1.5)         # balck base-link
ax.plot([0, 0], [0, -0.5], 'b--', lw=1.5, alpha=0.5)        # link-1 axis
extension_line, = ax.plot([], [], 'b--', lw=1.5, alpha=0.5) # link-2 axis

rods, = ax.plot([], [], 'o-', lw=1, color=colors[0], markersize=4) # double pendulum
trace, = ax.plot([], [], '-', lw=1, color='grey', alpha=0.5)       # trajectory trace

x, y = [], []

def init():
    rods.set_data([], [])
    trace.set_data([], [])
    extension_line.set_data([], [])

    x.clear()
    y.clear()
    
    return rods, trace, extension_line

def animate(i):
    q1 = states[0, i]
    q2 = states[1, i]

    # Forward Kinematics
    x1 = l1 * np.sin(q1)
    y1 = -l1 * np.cos(q1)
    x2 = x1 + l2 * np.sin(q1 + q2)
    y2 = y1 - l2 * np.cos(q1 + q2)

    rods.set_data([0, x1, x2], [0, y1, y2])
    
    x.append(x2)
    y.append(y2)
    trace.set_data(x[-100:], y[-100:])
    
    x_ext = x1 + 1/2 * np.sin(q1)
    y_ext = y1 - 1.2 * np.cos(q1)
    extension_line.set_data([x1, x_ext], [y1, y_ext])

    return rods, trace, extension_line

# Create animation
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=S, interval=50, blit=True)
anim.save("3-TVLQR-without-noise.mp4", writer='ffmpeg', fps=20, dpi=100)
plt.close()
HTML(anim.to_jshtml())

In [ ]:
# --- Cell 13 ---
# Diagrams
q1 = xs[:, 0]
q2 = xs[:, 1]
u1 = us[:, 0]
u2 = us[:, 1]
time = np.linspace(0, tf, S)

fig, axs = plt.subplots(2, 2, figsize=(14, 10))

# Angle of link 1
axs[0,0].set_title('Optimal and Actual Trajectory of Link-1')
axs[0,0].axhline(y=3.1415, color='g', linestyle='--', linewidth=0.8, alpha=0.5, label='$q_{1, goal}$')
axs[0,0].plot(time, q1_bar, linewidth=0.8, label='$q_{1, optimal}$')
axs[0,0].plot(time, q1, linewidth=0.8, label='$q_{1, actual}$')
axs[0,0].set_xlabel('Time (sec)')
axs[0,0].set_ylabel('Angle (rad)')
axs[0,0].legend()

# Angle of link 2
axs[0,1].set_title('Optimal and Actual Trajectory of Link-2')
axs[0,1].axhline(y=0, color='m', linestyle='--', linewidth=0.8, alpha=0.5, label='$q_{2, goal}$')
axs[0,1].plot(time, q2_bar, linewidth=0.8, label='$q_{2, optimal}$')
axs[0,1].plot(time, q2, linewidth=0.8, label='$q_{2, actual}$')
axs[0,1].set_xlabel('Time (sec)')
axs[0,1].set_ylabel('Angle (rad)')
axs[0,1].legend()

# Torque of link 1
axs[1,0].set_title('Torque of link 1')
axs[1,0].axhline(y=10, color='k', linestyle='--', linewidth=0.8, alpha=0.5, label='$u_{upper}$')
axs[1,0].axhline(y=-10, color='r', linestyle='--', linewidth=0.8, alpha=0.5, label='$u_{lower}$')
axs[1,0].plot(time, u1_bar, linewidth=0.8, label='$u_{1, optimal}$')
axs[1,0].plot(time, u1, linewidth=0.8, label='$u_{1, actual}$')
axs[1,0].set_xlabel('Time (sec)')
axs[1,0].set_ylabel('Torque (Nm)')
axs[1,0].legend()

# Torque of link 2
axs[1,1].set_title('Torque of link 2')
axs[1,1].axhline(y=10, color='k', linestyle='--', linewidth=0.8, alpha=0.5, label='$u_{upper}$')
axs[1,1].axhline(y=-10, color='r', linestyle='--', linewidth=0.8, alpha=0.5, label='$u_{lower}$')
axs[1,1].plot(time, u2_bar, linewidth=0.8, label='$u_{2, optimal}$')
axs[1,1].plot(time, u2, linewidth=0.8, label='$u_{2, actual}$')
axs[1,1].set_xlabel('Time (sec)')
axs[1,1].set_ylabel('Torque (Nm)')
axs[1,1].legend()

plt.savefig('3-diagrams-without-noise.png', dpi=300)

plt.tight_layout()
plt.show()

2. Add noise to the observations (Gaussian with zero mean and diagonal covariance). How does
increasing the noise affect the controller?

In [ ]:
# --- Cell 15 ---
# Initial state
x0 = jnp.array([[0., 0., 0., 0.]]).T

# Results
xs, us = my_controller(x0, add_noise=True)
xs = np.array(xs).reshape(-1,Ns)
us = np.array(us).reshape(-1,Nc)

In [ ]:
# --- Cell 13 ---
q1 = xs[:, 0]
q2 = xs[:, 1]
u1 = us[:, 0]
u2 = us[:, 1]
time = np.linspace(0, tf, S)

fig, axs = plt.subplots(2, 2, figsize=(14, 10))

# Angle of link 1
axs[0,0].set_title('Optimal and Actual Trajectory of Link-1 ()')
axs[0,0].axhline(y=3.1415, color='g', linestyle='--', linewidth=0.8, alpha=0.5, label='$q_{1, goal}$')
axs[0,0].plot(time, q1_bar, linewidth=0.8, label='$q_{1, optimal}$')
axs[0,0].plot(time, q1, linewidth=0.8, label='$q_{1, actual}$')
axs[0,0].set_xlabel('Time (sec)')
axs[0,0].set_ylabel('Angle (rad)')
axs[0,0].legend()

# Angle of link 2
axs[0,1].set_title('Optimal and Actual Trajectory of Link-2')
axs[0,1].axhline(y=0, color='m', linestyle='--', linewidth=0.8, alpha=0.5, label='$q_{2, goal}$')
axs[0,1].plot(time, q2_bar, linewidth=0.8, label='$q_{2, optimal}$')
axs[0,1].plot(time, q2, linewidth=0.8, label='$q_{2, actual}$')
axs[0,1].set_xlabel('Time (sec)')
axs[0,1].set_ylabel('Angle (rad)')
axs[0,1].legend()

# Torque of link 1
axs[1,0].set_title('Torque of link 1')
axs[1,0].axhline(y=10, color='k', linestyle='--', linewidth=0.8, alpha=0.5, label='$u_{upper}$')
axs[1,0].axhline(y=-10, color='r', linestyle='--', linewidth=0.8, alpha=0.5, label='$u_{lower}$')
axs[1,0].plot(time, u1_bar, linewidth=0.8, label='$u_{1, optimal}$')
axs[1,0].plot(time, u1, linewidth=0.8, label='$u_{1, actual}$')
axs[1,0].set_xlabel('Time (sec)')
axs[1,0].set_ylabel('Torque (Nm)')
axs[1,0].legend()

# Torque of link 2
axs[1,1].set_title('Torque of link 2')
axs[1,1].axhline(y=10, color='k', linestyle='--', linewidth=0.8, alpha=0.5, label='$u_{upper}$')
axs[1,1].axhline(y=-10, color='r', linestyle='--', linewidth=0.8, alpha=0.5, label='$u_{lower}$')
axs[1,1].plot(time, u2_bar, linewidth=0.8, label='$u_{2, optimal}$')
axs[1,1].plot(time, u2, linewidth=0.8, label='$u_{2, actual}$')
axs[1,1].set_xlabel('Time (sec)')
axs[1,1].set_ylabel('Torque (Nm)')
axs[1,1].legend()

plt.tight_layout()
plt.savefig('3-diagrams-with-noise.png', dpi=300)
plt.show()